In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
import pyemu
from pathlib import Path
import os, shutil
import flopy as fp

In [ ]:
ies_dir = Path('../tmprun/')
pst = pyemu.Pst(str(ies_dir / 'mv.ies.pst'))

In [ ]:
pst.parameter_data

In [ ]:
obs_true = pst.observation_data
obs_true

## take a look at the prior MC 

In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
phi0 = phi[0]

In [ ]:
phi0.hist(bins=50)


## let's check out the observations from prior MC and iES

In [ ]:
pst = pyemu.Pst(str(ies_dir / 'mv.ies.pst'))

In [ ]:
obsdat = pst.observation_data.copy()
obsdat.sample(5)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,5))
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[0])
phi.loc['base'].plot(color='b', ax=ax[0])
ax[0].set_yscale('log')
ax[0].set_title('log-scale $\\Phi$')
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[1])
phi.loc['base'].plot(color='b', ax=ax[1])
ax[1].set_title('linear-scale $\\Phi$');


In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
for cc in phi.columns:
    phi.loc[phi[cc]>1e6,cc] = np.nan

## We can focus in on iteration 3 as "best" and see how things changed due to iES

In [ ]:
citer = 2
obs = pd.read_csv(ies_dir / 'mv.ies.0.obs.csv', index_col=0)
obs_final = pd.read_csv(ies_dir / f'mv.ies.{citer}.obs.csv', index_col=0)

In [ ]:
with PdfPages(f'allobs.iter_{citer}.pdf') as outpdf:
    for cc in obs.columns:
        plt.figure()
        ax = obs[cc].hist(bins=20, color='grey', alpha=.5, density=1)
        obs_final[cc].hist(bins=20, color='blue', density=1)
        ax.axvline(obsdat.loc[cc].obsval, color='orange')
        plt.title(cc)
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

## how does this look spatially?

In [ ]:
# load up the model with flopy
sim = fp.mf6.MFSimulation.load(sim_ws = '../tmprun/')
m = sim.get_model()

In [ ]:
df_deep = pd.DataFrame.from_records(m.obs[2].continuous.data['at.csv'])
df_deep

In [ ]:
df_shallow = pd.DataFrame.from_records(m.obs[2].continuous.data['at.wt.csv'])
df_shallow

## find the cell centers for the head observations for plotting

In [ ]:
xc, yc = m.modelgrid.xcellcenters, m.modelgrid.ycellcenters
for df in [df_deep, df_shallow]:
    df['y'] = [yc[i[1],i[2]] for i in df['id']]
    df['x'] = [xc[i[1],i[2]] for i in df['id']]
    

In [ ]:
obs_final

## let's get the standard deviation of the residuals

In [ ]:
for df in [df_deep, df_shallow]:
    df['resid_prior_std'] = [(obs[i]-obs_true.loc[i].obsval).std() for i in df.obsname]
    df['resid_post_std'] = [(obs_final[i]-obs_true.loc[i].obsval).std() for i in df.obsname]
    df['base_prior_resid'] = [obs.loc['base',i]-obs_true.loc[i].obsval for i in df.obsname]
    df['base_post_resid'] = [obs_final.loc['base',i]-obs_true.loc[i].obsval for i in df.obsname]
    

In [ ]:
df

In [ ]:
fig, allax = plt.subplots(1,2, figsize=(10, 6))
for ax, cc in zip(allax,['base_prior_resid','base_post_resid']):
    ax.set_aspect('equal')
    pmv = fp.plot.PlotMapView(m, ax=ax)
    lc = pmv.plot_grid(lw=0.5)
    # pmv.plot_array(m.npf.k[1].array, alpha=.1)
    pmv.plot_bc("RIV")
    c = ax.scatter(df_shallow.x,df_shallow.y, s=df_shallow[cc]*200,c=df_shallow[cc],
                  norm=mpl.colors.CenteredNorm(), cmap='coolwarm')
    plt.colorbar(c)
    ax.set_title(cc.split('_')[1].capitalize(), fontsize=12)
plt.suptitle('Base Realization Shallow Residuals', fontsize=14)

In [ ]:
fig, allax = plt.subplots(1,2, figsize=(10, 6))
for ax, cc in zip(allax,['base_prior_resid','base_post_resid']):
    ax.set_aspect('equal')
    pmv = fp.plot.PlotMapView(m, ax=ax)
    lc = pmv.plot_grid(lw=0.5)
    # pmv.plot_array(m.npf.k[1].array, alpha=.1)
    pmv.plot_bc("RIV")
    c = ax.scatter(df_deep.x,df_deep.y, s=df_deep[cc]*200,c=df_deep[cc],
                  norm=mpl.colors.CenteredNorm(), cmap='coolwarm')
    plt.colorbar(c)
    ax.set_title(cc.split('_')[1].capitalize(), fontsize=12)
plt.suptitle('Base Realization Deep Residuals', fontsize=14)

In [ ]:
fig, allax = plt.subplots(1,2, figsize=(10, 6))
for ax, cc in zip(allax,['resid_prior_std','resid_post_std']):
    ax.set_aspect('equal')
    pmv = fp.plot.PlotMapView(m, ax=ax)
    lc = pmv.plot_grid(lw=0.5)
    pmv.plot_array(m.npf.k[1].array, alpha=.3)
    pmv.plot_bc("RIV")
    pmv.plot_bc("WEL", color='k')
    
    c = ax.scatter(df_shallow.x,df_shallow.y, s=df_shallow[cc]*100,c=df_shallow[cc])
    plt.colorbar(c)
    ax.set_title(cc.split('_')[1].capitalize(), fontsize=12)
plt.suptitle('Standard Deviation of Shallow Residuals', fontsize=14)
    

In [ ]:
fig, allax = plt.subplots(1,2, figsize=(10, 6))
for ax, cc in zip(allax,['resid_prior_std','resid_post_std']):
    ax.set_aspect('equal')
    pmv = fp.plot.PlotMapView(m, ax=ax)
    lc = pmv.plot_grid(lw=0.5)
    pmv.plot_array(m.npf.k[1].array, alpha=.3)
    pmv.plot_bc("RIV")
    c = ax.scatter(df_deep.x,df_deep.y, s=df_deep[cc]*100,c=df_deep[cc])
    plt.colorbar(c)
    ax.set_title(cc.split('_')[1].capitalize(), fontsize=12)
plt.suptitle('Standard Deviation of Deep Residuals', fontsize=14)
    

# Visualize a single set of fields

In [ ]:
tmpdir = Path('../scratch')
if tmpdir.exists():
    shutil.rmtree(tmpdir)
shutil.copytree(ies_dir, tmpdir)

In [ ]:
pst = pyemu.Pst(str(tmpdir / 'mv.ies.pst'))

In [ ]:
pars = pd.read_csv(ies_dir / 'mv.ies.3.par.csv', index_col=0)
pars

In [ ]:
pst.control_data.noptmax=0

## good to check out the base realization, but could choose another

In [ ]:
real_to_plot = 'base'

In [ ]:
pst.parameter_data.loc[pars.loc[real_to_plot].index, 'parval1'] = pars.loc[real_to_plot]

In [ ]:
pst.write(str(tmpdir / 'testrun.pst'), version=2)

In [ ]:
pyemu.os_utils.run('pestpp-ies testrun.pst', str(tmpdir))

## viz the k fields and other parameters

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10))
for cax, carr, ctitle in zip(ax.ravel(), [np.loadtxt(tmpdir / 'k_aq.ref'), m.npf.k.data[1],
                          np.loadtxt(tmpdir / 'k_clay.ref'), m.npf.k.data[2]],
                         ['Estimated Aquifer K','True Aquifer K','Estimated Clay K','True Clay K']):
    cax.set_aspect('equal')
    pmv = fp.plot.PlotMapView(m, ax=cax)
    lc = pmv.plot_grid(lw=0.5)
    c = pmv.plot_array(carr)
    plt.colorbar(c)
    cax.set_title(ctitle)

In [ ]:
pars.kaniso.hist(bins=50)
plt.title('Anisotropy: aquifer');

In [ ]:
pars.kaniso_clay.hist(bins=50)
plt.title('Anisotropy: clay layer');

In [ ]:
pars.rch.hist(bins=50)
plt.title('Recharge');